In [ ]:
import numpy as np
import flammkuchen as fl
import napari
from pathlib import Path

from split_dataset import SplitDataset
import json

import tifffile as tiff
import flammkuchen as fl 

In [ ]:
master =  Path(r"Z:\Hagar and Ot\e0075\ipn h2b")
fish_list = list(master.glob("*_f*"))
fish = fish_list[0]

In [ ]:
anat = tiff.imread(fish / "anatomy_from_suite2p.tif")

In [ ]:
viewer = napari.view_image(anat,  colormap="gray")
region = np.zeros(np.shape(anat), dtype=int)

labels_layer = viewer.add_labels(region, name='region')

In [ ]:
#### Run the next cells only after marking the regions you want to keep in napari

In [ ]:
labels = viewer.layers["region"].data
d = {
    'ipn_mask': labels,
}
fl.save(fish / 'ipn_mask.h5', d)

In [ ]:
### go over each plane and keep coords within mask
#loading the mask:
labels = fl.load(fish / 'ipn_mask.h5')['ipn_mask']

In [ ]:
viewer = napari.view_image(anat,  colormap="gray")
labels_layer = viewer.add_labels(labels)


In [ ]:
master =  Path(r"Z:\Hagar\e0075\ablation\control")
fish_list = list(master.glob("*_pre*"))

In [ ]:
for fish in fish_list:
    print(fish)
    planes_path = fish / "suite2p"
    planes = list(planes_path.glob("*00*"))
    
    #loading the mask:
    labels = fl.load(fish / 'ipn_mask.h5')['ipn_mask']
    count = 0
    for plane in planes:

        curr_mask = labels[count]
        try:
            suite2p_data = fl.load(plane / 'data_from_suite2p_cells.h5')
        except:
            suite2p_data = fl.load(plane / 'data_from_suite2p_unfiltered.h5')
            
        all_coords = suite2p_data['coords']

        coords_to_remove = []
        coords_to_keep = []
        num_cells = np.shape(all_coords)[0]
        for cell in range(num_cells):
            tmp_cell = labels[int(all_coords[cell, 0]), int(all_coords[cell, 1]), int(all_coords[cell, 2])]
            tmp_cell = curr_mask[int(all_coords[cell, 1]), int(all_coords[cell, 2])]
            if tmp_cell > 0:
                coords_to_keep = coords_to_keep + [cell]
            else:
                coords_to_remove = coords_to_remove + [cell]

        count += 1
        d = {
            'coords_to_keep': coords_to_keep,
        }
        fl.save(plane / 'ipn_coords.h5', d)